In [1]:
import os
import sys

sys.path.append('')

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from pathlib import Path

import torch
import torch.nn as nn
from torchinfo import summary

from baard.classifiers.mnist_cnn import MNIST_CNN

In [4]:
PATH_ROOT = Path(os.getcwd()).parent
PATH_DATA = os.path.join(PATH_ROOT, 'data')
PATH_CHECKPOINT = os.path.join(PATH_ROOT, 'pretrained_clf', 'mnist_cnn.ckpt')
print(PATH_CHECKPOINT)

/home/lukec/workspace/baard_v4/pretrained_clf/mnist_cnn.ckpt


In [5]:
model = MNIST_CNN.load_from_checkpoint(PATH_CHECKPOINT)
batch_size = model.train_dataloader().batch_size
input_size = (batch_size, 1, 28, 28)
device = 'cpu'

results = summary(model, input_size=input_size, device=device)
print(results)

Layer (type:depth-idx)                   Output Shape              Param #
MNIST_CNN                                [256, 10]                 --
├─Conv2d: 1-1                            [256, 32, 26, 26]         320
├─ReLU: 1-2                              [256, 32, 26, 26]         --
├─Conv2d: 1-3                            [256, 64, 24, 24]         18,496
├─ReLU: 1-4                              [256, 64, 24, 24]         --
├─MaxPool2d: 1-5                         [256, 64, 12, 12]         --
├─Flatten: 1-6                           [256, 9216]               --
├─Linear: 1-7                            [256, 200]                1,843,400
├─ReLU: 1-8                              [256, 200]                --
├─Linear: 1-9                            [256, 10]                 2,010
Total params: 1,864,226
Trainable params: 1,864,226
Non-trainable params: 0
Total mult-adds (G): 3.26
Input size (MB): 0.80
Forward/backward pass size (MB): 120.23
Params size (MB): 7.46
Estimated Total Size (M

In [6]:
list(model.children())

[Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1)),
 ReLU(),
 Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1)),
 ReLU(),
 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
 Flatten(start_dim=1, end_dim=-1),
 Linear(in_features=9216, out_features=200, bias=True),
 ReLU(),
 Linear(in_features=200, out_features=10, bias=True)]

In [7]:
print(len(list(model.children())))

9


In [8]:
latent_net1 = nn.Sequential(*list(model.children())[:3])

summary(latent_net1, input_size=input_size, device=device)

Layer (type:depth-idx)                   Output Shape              Param #
Sequential                               [256, 64, 24, 24]         --
├─Conv2d: 1-1                            [256, 32, 26, 26]         320
├─ReLU: 1-2                              [256, 32, 26, 26]         --
├─Conv2d: 1-3                            [256, 64, 24, 24]         18,496
Total params: 18,816
Trainable params: 18,816
Non-trainable params: 0
Total mult-adds (G): 2.78
Input size (MB): 0.80
Forward/backward pass size (MB): 119.80
Params size (MB): 0.08
Estimated Total Size (MB): 120.68

In [9]:
latent_net2 = nn.Sequential(*list(model.children())[:6])

summary(latent_net2, input_size=input_size, device=device)

Layer (type:depth-idx)                   Output Shape              Param #
Sequential                               [256, 9216]               --
├─Conv2d: 1-1                            [256, 32, 26, 26]         320
├─ReLU: 1-2                              [256, 32, 26, 26]         --
├─Conv2d: 1-3                            [256, 64, 24, 24]         18,496
├─ReLU: 1-4                              [256, 64, 24, 24]         --
├─MaxPool2d: 1-5                         [256, 64, 12, 12]         --
├─Flatten: 1-6                           [256, 9216]               --
Total params: 18,816
Trainable params: 18,816
Non-trainable params: 0
Total mult-adds (G): 2.78
Input size (MB): 0.80
Forward/backward pass size (MB): 119.80
Params size (MB): 0.08
Estimated Total Size (MB): 120.68

In [10]:
latent_net3 = nn.Sequential(*list(model.children())[:7])

summary(latent_net3, input_size=input_size, device=device)

Layer (type:depth-idx)                   Output Shape              Param #
Sequential                               [256, 200]                --
├─Conv2d: 1-1                            [256, 32, 26, 26]         320
├─ReLU: 1-2                              [256, 32, 26, 26]         --
├─Conv2d: 1-3                            [256, 64, 24, 24]         18,496
├─ReLU: 1-4                              [256, 64, 24, 24]         --
├─MaxPool2d: 1-5                         [256, 64, 12, 12]         --
├─Flatten: 1-6                           [256, 9216]               --
├─Linear: 1-7                            [256, 200]                1,843,400
Total params: 1,862,216
Trainable params: 1,862,216
Non-trainable params: 0
Total mult-adds (G): 3.25
Input size (MB): 0.80
Forward/backward pass size (MB): 120.21
Params size (MB): 7.45
Estimated Total Size (MB): 128.46

In [11]:
import torch

a = torch.rand(10, 3, 3, 3)
a = a.flatten(start_dim=1)
print(a.size())

b = torch.rand(10, 9)
b = b.flatten(start_dim=1)
print(b.size())

torch.Size([10, 27])
torch.Size([10, 9])


In [12]:
torch.hstack([a, b]).size()

torch.Size([10, 36])

In [13]:
import numpy as np
from torch import Tensor

In [14]:
def create_loo_input(x: Tensor) -> Tensor:
    """Create a LOO mapping for 1 example. Output is in shape of (n_features + 1, n_features).
    It includes masking 1 input with 0 for every feature + the original input.
    NOTE: Input is expected to be a single example, not a list.
    """
    # Get the actual number of input features
    n_features = int(torch.Tensor(list(x.size())).prod().item())
    # The output should be (n_features + 1, <ORIGINAL_SAMPLE_SHAPE>)
    shape_repeat = [n_features + 1] + [1 for _ in range(len(x.size()))]
    # Add additional dimension
    X_repeated = x.unsqueeze(0).repeat(shape_repeat)

    # Create a mask with 0 for each pixel. Keep the last example original.
    X_loo_mask = torch.ones_like(X_repeated)
    loo_shape = X_loo_mask.size()
    X_loo_mask = X_loo_mask.reshape(len(X_loo_mask),-1)
    for i in range(X_loo_mask.size(-1)):
        X_loo_mask[i, i] = 0
    X_loo_mask = X_loo_mask.reshape(loo_shape)

    X_loo = X_loo_mask * X_repeated
    return X_loo

In [15]:
# x = torch.Tensor([[1, 2], [3, 4]])
x = torch.rand(1, 28, 28)

X_loo = create_loo_input(x)

print(X_loo)

tensor([[[[0.0000, 0.9996, 0.9704,  ..., 0.8242, 0.4138, 0.6291],
          [0.2083, 0.3695, 0.1850,  ..., 0.4992, 0.2961, 0.0463],
          [0.0150, 0.0673, 0.5763,  ..., 0.2165, 0.6689, 0.3881],
          ...,
          [0.9172, 0.7462, 0.4110,  ..., 0.8639, 0.9305, 0.2461],
          [0.6870, 0.1068, 0.6394,  ..., 0.2540, 0.0146, 0.7522],
          [0.2638, 0.3727, 0.9201,  ..., 0.5644, 0.4115, 0.0024]]],


        [[[0.0515, 0.0000, 0.9704,  ..., 0.8242, 0.4138, 0.6291],
          [0.2083, 0.3695, 0.1850,  ..., 0.4992, 0.2961, 0.0463],
          [0.0150, 0.0673, 0.5763,  ..., 0.2165, 0.6689, 0.3881],
          ...,
          [0.9172, 0.7462, 0.4110,  ..., 0.8639, 0.9305, 0.2461],
          [0.6870, 0.1068, 0.6394,  ..., 0.2540, 0.0146, 0.7522],
          [0.2638, 0.3727, 0.9201,  ..., 0.5644, 0.4115, 0.0024]]],


        [[[0.0515, 0.9996, 0.0000,  ..., 0.8242, 0.4138, 0.6291],
          [0.2083, 0.3695, 0.1850,  ..., 0.4992, 0.2961, 0.0463],
          [0.0150, 0.0673, 0.5763,  ..

In [16]:
X_loo.size()

torch.Size([785, 1, 28, 28])

In [17]:
np.prod([1, 28, 28])

784

In [18]:
combined_features = {data_type: {} for data_type in ['test', 'train']}
combined_features

{'test': {}, 'train': {}}

In [19]:
X = np.array([[[0.8, 0.2, 0.4, 0.4],
               [0.1, 0.3, 0.9, 0.3],
               [0.5, 0.7, 0.5, 0.1]]])
X.mean(2)

array([[0.45, 0.4 , 0.45]])

In [20]:
probs = np.array([
    [0.84480608, 0.15519392],
    [0.1463647, 0.8536353],
    [0.7195677, 0.2804323],
    [0.1463647, 0.8536353],
    [0.7195677, 0.2804323],
    [0.1463647, 0.8536353],
    [0.7195677, 0.2804323],
    [0.55334507, 0.44665493],
    [0.17953881, 0.82046119],
    [0.31972605, 0.68027395],
])
probs[:, 1]

array([0.15519392, 0.8536353 , 0.2804323 , 0.8536353 , 0.2804323 ,
       0.8536353 , 0.2804323 , 0.44665493, 0.82046119, 0.68027395])